In [28]:




!pip install langchain openai faiss-cpu spyder-kernels
!pip install -U langchain-community



In [29]:
# The libraries we need
import os, sys
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document

In [30]:
# Put your API key here
OPENAI_API_KEY = ""
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [31]:
# 1) Build a tiny KB
docs = [
    Document(page_content="Paris is the capital of France",  metadata={"src": "wiki"}),
    Document(page_content="Berlin is the capital of Germany", metadata={"src": "wiki"}),
]


In [32]:
# 1) Instantiate the embedder
embedder = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=OPENAI_API_KEY
)

# 2) Build the FAISS index using that embedder
store = FAISS.from_documents(docs, embedder)


In [33]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",          # ← same as LangChain’s default
    temperature=0,
    openai_api_key=OPENAI_API_KEY
)


In [ ]:
# 5) RAG function
def rag(question: str, k: int = 2) -> str:
    results = store.similarity_search(question, k)
    ctx = "\n".join(d.page_content for d in results)
    prompt = f"Answer using only this context:\n{ctx}\n\nQ: {question}\nA:"
    return llm.predict(prompt)

In [ ]:
# 6) Interactive loop
if __name__ == "__main__":
    print("Type ‘exit’ to quit")
    while True:
        q = input("Your question: ")
        if q.lower() in ("exit", "quit"):
            sys.exit()
        print("Answer:", rag(q))

Type ‘exit’ to quit
